In [1]:
# If gspx is not installed, we add it to the path
import os, sys
gdir = os.path.dirname(os.getcwd())  # parent folder
sys.path.insert(0, gdir)

In [2]:
import matplotlib.pyplot as plt
import numpy as np

from gspx.utils.graph import make_sensor
from gspx.utils.display import plot_graph
from gspx.signals import QuaternionSignal
from gspx.qgsp import QMatrix
from gspx.adaptive import QLMS
from gspx.utils.quaternion_matrix import explode_quaternions
from gspx.datasets import WeatherGraphData, uk_weather

In [4]:
df = uk_weather()
df

,town,latitude,longitude,humidity,pressure,temp,wind_speed,wind_degrees
0,"Troon, Scotland, the UK",55.540001,-4.660000,55,1018,289.19,3.60,220
1,"St.Asaph, Wales, the UK",53.257999,-3.442000,59,1017,288.49,3.71,99
2,"Stirling, Scotland, UK",56.116600,-3.936900,53,1018,288.15,3.72,114
3,"Welling, Bexley, London, the UK",51.456600,0.105600,39,1014,290.09,8.23,80
4,"Bonnybridge, Scotland, UK",56.003227,-3.888634,50,1018,288.00,3.53,106
...,...,...,...,...,...,...,...,...
172,"Newport, Telford and Wrekin, UK",52.768555,-2.375587,42,1014,288.61,6.17,110
173,"Dover, Kent, UK",51.126369,1.316198,52,1015,288.26,7.95,56
174,"Limavady, UK",55.051434,-6.950054,44,1016,289.02,7.20,190
175,"Litcham, Norfolk, UK",52.724182,0.791488,45,1017,289.58,5.14,90


In [3]:
uk_data = WeatherGraphData()
A, coords = uk_data.graph
s = uk_data.signal

In [ ]:
N = 6
A1, coords = make_sensor(N, seed=2)
Ai, _ = make_sensor(N, seed=3)
Aj, _ = make_sensor(N, seed=4)
Ak, _ = make_sensor(N, seed=5)

X = QMatrix([A1, Ai, Aj, Ak])

In [ ]:
self = QLMS()

m, d = X.shape
X_ = QLMS.add_intercept_term(X)
X_, mu, std = QLMS.normal_scaling(X_, ignore_first=True)

self.mu_ = mu
self.std_ = std

In [ ]:
theta = QLMS.initiate(length=d + 1, method='zeros')
J = np.zeros(self.max_iter)

err = X_ * theta - y

In [ ]:
res = {}
for lr in self.alpha:
    theta = QLMS.initiate(length=d + 1, method='zeros')
    J = np.zeros(self.max_iter)

    for idx_iter in range(self.max_iter):
        err = X_ @ theta - y

        # Calculate the J term, which is the current MSE
        J[idx_iter] = (0.5/m) * err.T @ err

        # The gradient
        grad = (1/m) * np.conjugate(X_).T @ err

        # Here is the actual update
        theta = theta - lr * grad
    res[lr] = dict(result=theta, cost=J)

self.res_ = res
idx_lower_cost = np.argmin([v['cost'][-1] for k, v in res.items()])
self.best_lr_ = self.alpha[idx_lower_cost]

return self